# Problem Statement : Hospital Patient Data Analysis

### Context:#### 
A hospital maintains patient records including admission details, department, diagnosis, doctor, and bill amount. You have two datasets: one wit  patient info and another with billing details. Some patients have blank bill amounts, and there are multiple rows for the same patient due to follow-ups.


In [7]:
import pandas as pd
import numpy as np


In [9]:
# Load patient data
patient_df = pd.read_csv("Patient_Data.csv")

# Load billing data
billing_df = pd.read_csv("Billing_Data.csv")


### Task 1: Show Patient Dataset Summary

In [14]:
patient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes


### Task 2: Select Billing-Relevant Columns

In [19]:
billing_columns = ['PatientID', 'Department', 'Doctor', 'BillAmount']
patient_billing_df = patient_df[billing_columns]

patient_billing_df.head()


,PatientID,Department,Doctor,BillAmount
0,101,Cardiology,Dr. Smith,5000.0
1,102,Neurology,Dr. John,NaN
2,103,Orthopedics,Dr. Lee,7500.0
3,104,Cardiology,Dr. Smith,6200.0
4,105,Dermatology,Dr. Rose,NaN


### Task 3: Drop Administrative Columns

In [22]:
patient_df = patient_df.drop(columns=['ReceptionistID', 'CheckInTime'])
patient_df.head()


,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,NaN


### Task 4: Total Bill Amount per Department

In [25]:
department_revenue = patient_billing_df.groupby('Department')['BillAmount'].sum()
department_revenue


Department
Cardiology     16200.0
Dermatology        0.0
Neurology          0.0
Orthopedics     7500.0
Name: BillAmount, dtype: float64

### Task 5: Remove Duplicate Patient Records

In [28]:
patient_df = patient_df.drop_duplicates(subset='PatientID')
patient_df.shape


(5, 5)

### Task 6: Fill Missing BillAmount with Mean

In [31]:
mean_bill = patient_billing_df['BillAmount'].mean()
patient_billing_df['BillAmount'] = patient_billing_df['BillAmount'].fillna(mean_bill)

patient_billing_df.isnull().sum()


PatientID     0
Department    0
Doctor        0
BillAmount    0
dtype: int64

### Task 7: Merge Billing & Patient Datasets

In [34]:
merged_df = pd.merge(patient_df, billing_df, on='PatientID', how='inner')
merged_df.head()


,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0,2000,3000
1,102,Bob,Neurology,Dr. John,NaN,1500,3500
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,2500,5000
3,104,David,Cardiology,Dr. Smith,6200.0,3000,3200
4,105,Eva,Dermatology,Dr. Rose,NaN,1000,4000


### Task 8: Concatenate New Patient Records (Row-wise)

In [37]:
# New patient data (current week)
new_patients = pd.DataFrame({
    'PatientID': [201, 202],
    'Department': ['Cardiology', 'Neurology'],
    'Doctor': ['Dr. Mehta', 'Dr. Rao'],
    'BillAmount': [45000, 38000]
})

# Row-wise concatenation
merged_df = pd.concat([merged_df, new_patients], axis=0, ignore_index=True)
merged_df.tail()


,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,2500.0,5000.0
3,104,David,Cardiology,Dr. Smith,6200.0,3000.0,3200.0
4,105,Eva,Dermatology,Dr. Rose,NaN,1000.0,4000.0
5,201,NaN,Cardiology,Dr. Mehta,45000.0,NaN,NaN
6,202,NaN,Neurology,Dr. Rao,38000.0,NaN,NaN


### Task 9: Concatenate New Billing Columns (Column-wise)

In [40]:
# Additional billing details
billing_extra = pd.DataFrame({
    'InsuranceCovered': ['Yes'] * len(merged_df),
    'FinalAmount': merged_df['BillAmount'] * 0.9
})

# Column-wise concatenation
final_df = pd.concat([merged_df, billing_extra], axis=1)
final_df.head()


,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0,2000.0,3000.0,Yes,4500.0
1,102,Bob,Neurology,Dr. John,NaN,1500.0,3500.0,Yes,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,2500.0,5000.0,Yes,6750.0
3,104,David,Cardiology,Dr. Smith,6200.0,3000.0,3200.0,Yes,5580.0
4,105,Eva,Dermatology,Dr. Rose,NaN,1000.0,4000.0,Yes,NaN
